# 데이터 처리

- 중복된 데이터 제거
- 데이터 타입 변환(물량, 금액을 숫자형으로)
- 중복된 행 제거
- 결측치 포함된 행 제거
- 컬럼 이름 앞뒤 공백 제거
- 품종, 법인 컬럼 제거
- 일자에서 연도, 월, 일 추출하여 별도 컬럼으로 추가
- 계절 컬럼 추가
- [일자, 부류, 품목, 도매시장] 기준으로 데이터 그룹화
- 총물량(kg), 평균물량(kg), 총금액(원), 평균금액(원), 총단가(원/kg), 평균단가(원/kg) 추가
- 총단가(원/kg): 총금액(원) ÷ 총물량(kg)
- 평균단가(원/kg): 평균금액(원) ÷ 평균물량(kg)
- 음수 값 제거
- 컬럼 순서 재정렬
- 데이터 병합 후 연도-계절별 데이터 분할 저장


In [1]:
import os
import pandas as pd
from glob import glob

# 입력 및 출력 디렉토리 설정
input_dir = r"../data/raw/agromarket"
processed_dir = r"../data/processed/agromarket_auction_price"
final_output_dir = r"../data/processed/agromarket_yearandseason"

# 출력 디렉토리 생성 (존재하지 않으면 생성)
os.makedirs(processed_dir, exist_ok=True)
os.makedirs(final_output_dir, exist_ok=True)


In [2]:
def load_and_preprocess_files(input_dir):
    file_paths = glob(os.path.join(input_dir, "*.xls"))
    merged_data = pd.DataFrame()

    for file_path in file_paths:
        try:
            # 파일 읽기 및 중복 제거
            data = pd.read_excel(file_path, engine='xlrd')
            data = data.drop_duplicates()

            # 데이터 타입 변환 및 결측치 처리
            for col in ["물량(kg)", "금액(원)"]:
                data[col] = pd.to_numeric(data.get(col, pd.Series()), errors='coerce')

            # 음수 값 제거
            data = data[(data["물량(kg)"] > 0) & (data["금액(원)"] > 0)]

            # 예상 컬럼 확인
            expected_columns = ["일자", "부류", "품목", "도매시장", "물량(kg)", "금액(원)"]
            if not set(expected_columns).issubset(data.columns):
                continue

            # 결측치 제거 및 '일자' 처리
            data = data.dropna()
            data["일자"] = pd.to_datetime(data["일자"], format="%Y%m%d", errors='coerce')
            data = data.dropna(subset=["일자"])
            data["연도"] = data["일자"].dt.year
            data["월"] = data["일자"].dt.month
            data["일"] = data["일자"].dt.day

            # 계절 정보 추가
            def get_season(month):
                if month in [12, 1, 2]:
                    return "겨울"
                elif month in [3, 4, 5]:
                    return "봄"
                elif month in [6, 7, 8]:
                    return "여름"
                else:
                    return "가을"

            data["계절"] = data["월"].apply(get_season)

            # 그룹화 및 집계 계산
            grouped_data = data.groupby(["일자", "연도", "월", "일", "계절", "부류", "품목", "도매시장"], as_index=False).agg({
                "물량(kg)": ["sum", "mean"],
                "금액(원)": ["sum", "mean"]
            })
            grouped_data.columns = [
                "일자", "연도", "월", "일", "계절", "부류", "품목", "도매시장",
                "총물량(kg)", "평균물량(kg)", "총금액(원)", "평균금액(원)"
            ]

            # 단가 계산 (0으로 나누기 방지)
            grouped_data["총단가(원/kg)"] = grouped_data["총금액(원)"] / grouped_data["총물량(kg)"].replace(0, pd.NA)
            grouped_data["평균단가(원/kg)"] = grouped_data["평균금액(원)"] / grouped_data["평균물량(kg)"].replace(0, pd.NA)

            # 결측치 처리 및 단가 음수 제거
            grouped_data = grouped_data.fillna(0)
            grouped_data = grouped_data[(grouped_data["총단가(원/kg)"] >= 0) & (grouped_data["평균단가(원/kg)"] >= 0)]

            # 병합
            merged_data = pd.concat([merged_data, grouped_data], ignore_index=True)

        except Exception as e:
            print(f"오류 발생: {file_path} - {e}")

    # 컬럼 순서 정리
    final_columns = [
        "일자", "연도", "월", "일", "계절", "부류", "품목", "도매시장",
        "총물량(kg)", "총금액(원)", "총단가(원/kg)", "평균물량(kg)", "평균금액(원)", "평균단가(원/kg)"
    ]
    return merged_data[final_columns]

# 전처리 실행
merged_data = load_and_preprocess_files(input_dir)

In [3]:
# 전처리된 데이터의 기본 정보 확인
print("전처리된 데이터 기본 정보:")
print(merged_data.info())

# 데이터 샘플 확인
print("\n전처리된 데이터 샘플:")
print(merged_data.head())

# 결측치 확인
print("\n결측치 확인:")
print(merged_data.isnull().sum())

# 숫자형 컬럼의 통계 정보 확인
print("\n숫자형 컬럼 통계:")
print(merged_data.describe())


전처리된 데이터 기본 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5643209 entries, 0 to 5643208
Data columns (total 14 columns):
 #   Column      Dtype         
---  ------      -----         
 0   일자          datetime64[ns]
 1   연도          int32         
 2   월           int32         
 3   일           int32         
 4   계절          object        
 5   부류          object        
 6   품목          object        
 7   도매시장        object        
 8   총물량(kg)     float64       
 9   총금액(원)      float64       
 10  총단가(원/kg)   float64       
 11  평균물량(kg)    float64       
 12  평균금액(원)     float64       
 13  평균단가(원/kg)  float64       
dtypes: datetime64[ns](1), float64(6), int32(3), object(4)
memory usage: 538.2+ MB
None

전처리된 데이터 샘플:
          일자    연도  월  일  계절   부류   품목 도매시장  총물량(kg)      총금액(원)    총단가(원/kg)  \
0 2019-01-03  2019  1  3  겨울  과실류   감귤   강릉  20293.0  34098700.0  1680.318336   
1 2019-01-03  2019  1  3  겨울  과실류   단감   강릉   1120.0   2886600.0  2577.321429   
2 2019-01-03  

In [4]:
def save_by_year_and_season(merged_data, output_dir):
    # 출력 디렉토리 생성 (존재하지 않으면 생성)
    os.makedirs(output_dir, exist_ok=True)

    # 계절 이름 매핑
    season_mapping = {"봄": "Spring", "여름": "Summer", "가을": "Autumn", "겨울": "Winter"}

    # 연도-계절별 데이터 분할 및 저장
    for year in merged_data["연도"].unique():
        for season_kor, season_eng in season_mapping.items():
            subset = merged_data[(merged_data["연도"] == year) & (merged_data["계절"] == season_kor)]
            if not subset.empty:
                output_file = os.path.join(output_dir, f"agromarket_{year}_{season_eng}.csv")
                subset.to_csv(output_file, index=False, encoding="utf-8-sig")
                print(f"파일 저장 완료: {output_file}")

# 저장 실행
output_dir = "../data/processed/agromarket_yearandseason"
save_by_year_and_season(merged_data, output_dir)

print("연도-계절별 데이터 저장 완료.")


파일 저장 완료: ../data/processed/agromarket_yearandseason\agromarket_2019_Spring.csv
파일 저장 완료: ../data/processed/agromarket_yearandseason\agromarket_2019_Summer.csv
파일 저장 완료: ../data/processed/agromarket_yearandseason\agromarket_2019_Autumn.csv
파일 저장 완료: ../data/processed/agromarket_yearandseason\agromarket_2019_Winter.csv
파일 저장 완료: ../data/processed/agromarket_yearandseason\agromarket_2020_Spring.csv
파일 저장 완료: ../data/processed/agromarket_yearandseason\agromarket_2020_Summer.csv
파일 저장 완료: ../data/processed/agromarket_yearandseason\agromarket_2020_Autumn.csv
파일 저장 완료: ../data/processed/agromarket_yearandseason\agromarket_2020_Winter.csv
파일 저장 완료: ../data/processed/agromarket_yearandseason\agromarket_2021_Spring.csv
파일 저장 완료: ../data/processed/agromarket_yearandseason\agromarket_2021_Summer.csv
파일 저장 완료: ../data/processed/agromarket_yearandseason\agromarket_2021_Autumn.csv
파일 저장 완료: ../data/processed/agromarket_yearandseason\agromarket_2021_Winter.csv
파일 저장 완료: ../data/processed/agromarket_y

In [5]:
# 전체 행 수와 컬럼 이름 확인
print(f"전체 데이터 행 수: {len(merged_data)}")
print("컬럼 이름:")
print(merged_data.columns.tolist())

전체 데이터 행 수: 5643209
컬럼 이름:
['일자', '연도', '월', '일', '계절', '부류', '품목', '도매시장', '총물량(kg)', '총금액(원)', '총단가(원/kg)', '평균물량(kg)', '평균금액(원)', '평균단가(원/kg)']


In [6]:
# 중복된 행 수 확인
duplicate_count = merged_data.duplicated().sum()
print(f"중복된 행 수: {duplicate_count}")


중복된 행 수: 0


In [7]:
# 음수 값이 포함된 행 필터링
negative_rows = merged_data[
    (merged_data["총물량(kg)"] < 0) |
    (merged_data["평균물량(kg)"] < 0) |
    (merged_data["총금액(원)"] < 0) |
    (merged_data["평균금액(원)"] < 0) |
    (merged_data["총단가(원/kg)"] < 0) |
    (merged_data["평균단가(원/kg)"] < 0)
]

print(f"음수 값이 포함된 행 수: {len(negative_rows)}")
print(negative_rows)


음수 값이 포함된 행 수: 0
Empty DataFrame
Columns: [일자, 연도, 월, 일, 계절, 부류, 품목, 도매시장, 총물량(kg), 총금액(원), 총단가(원/kg), 평균물량(kg), 평균금액(원), 평균단가(원/kg)]
Index: []


In [8]:
# NaN 또는 데이터가 없는 행 필터링
nan_rows = merged_data[merged_data.isnull().any(axis=1)]

# NaN이 포함된 행 수 출력
print(f"NaN이 포함된 행 수: {len(nan_rows)}")

# NaN이 포함된 행 출력
print(nan_rows)


NaN이 포함된 행 수: 0
Empty DataFrame
Columns: [일자, 연도, 월, 일, 계절, 부류, 품목, 도매시장, 총물량(kg), 총금액(원), 총단가(원/kg), 평균물량(kg), 평균금액(원), 평균단가(원/kg)]
Index: []


# 지역 컬럼 추가

In [2]:
import os
import pandas as pd
from glob import glob

# 디렉토리 경로 설정
file_path = "../data/processed/agromarket_yearandseason/"

# 디렉토리 내 모든 CSV 파일 읽어오기
all_files = glob(os.path.join(file_path, "*.csv"))

# 데이터 병합
merged_data = pd.concat([pd.read_csv(file) for file in all_files], ignore_index=True)

# 도매시장 고유 값 추출
unique_markets = merged_data["도매시장"].unique()
unique_markets


array(['강릉', '광주각화', '광주서부', '구리', '구미', '대구북부', '대전노은', '대전오정', '부산반여',
       '부산엄궁', '서울가락', '서울강서', '수원', '순천', '안동', '안산', '안양', '울산', '원주',
       '익산', '인천남촌', '인천삼산', '전주', '정읍', '진주', '창원내서', '창원팔용', '천안', '청주',
       '춘천', '충주', '포항', '부산국제수산'], dtype=object)

In [3]:
# 지역명 추출
def extract_region(market_name):
    return market_name[:2]  # 앞 두 글자 추출

# 지역 컬럼 추가
merged_data["지역"] = merged_data["도매시장"].apply(extract_region)

# 지역별 고유 값 확인
print("추출된 지역명:")
print(merged_data["지역"].unique())

추출된 지역명:
['강릉' '광주' '구리' '구미' '대구' '대전' '부산' '서울' '수원' '순천' '안동' '안산' '안양' '울산'
 '원주' '익산' '인천' '전주' '정읍' '진주' '창원' '천안' '청주' '춘천' '충주' '포항']


In [5]:
merged_data.columns

Index(['일자', '연도', '월', '일', '계절', '부류', '품목', '도매시장', '총물량(kg)', '총금액(원)',
       '총단가(원/kg)', '평균물량(kg)', '평균금액(원)', '평균단가(원/kg)', '지역'],
      dtype='object')

In [6]:
# 컬럼 순서 재정렬
reordered_columns = [
    '일자', '연도', '월', '일', '계절', '지역', '도매시장', '부류', '품목', '총물량(kg)', '총금액(원)', '총단가(원/kg)', '평균물량(kg)', '평균금액(원)', '평균단가(원/kg)'
]

# 컬럼 순서 변경
merged_data = merged_data[reordered_columns]
merged_data.head()

,일자,연도,월,일,계절,지역,도매시장,부류,품목,총물량(kg),총금액(원),총단가(원/kg),평균물량(kg),평균금액(원),평균단가(원/kg)
0,2019-09-02,2019,9,2,가을,강릉,강릉,과실류,감귤,650.0,2670000.0,4107.692308,650.0,2.670000e+06,4107.692308
1,2019-09-02,2019,9,2,가을,강릉,강릉,과실류,곶감,48.0,132000.0,2750.000000,48.0,1.320000e+05,2750.000000
2,2019-09-02,2019,9,2,가을,강릉,강릉,과실류,무화과,20.0,90000.0,4500.000000,20.0,9.000000e+04,4500.000000
3,2019-09-02,2019,9,2,가을,강릉,강릉,과실류,바나나,1404.0,2938000.0,2092.592593,702.0,1.469000e+06,2092.592593
4,2019-09-02,2019,9,2,가을,강릉,강릉,과실류,배,10657.5,34569400.0,3243.668778,3552.5,1.152313e+07,3243.668778


In [9]:
# 저장
def save_by_year_and_season(merged_data, output_dir):
    # 출력 디렉토리 생성 (존재하지 않으면 생성)
    os.makedirs(output_dir, exist_ok=True)

    # 계절 이름 매핑
    season_mapping = {"봄": "Spring", "여름": "Summer", "가을": "Autumn", "겨울": "Winter"}

    # 연도-계절별 데이터 분할 및 저장
    for year in merged_data["연도"].unique():
        for season_kor, season_eng in season_mapping.items():
            subset = merged_data[(merged_data["연도"] == year) & (merged_data["계절"] == season_kor)]
            if not subset.empty:  # 데이터가 있을 경우에만 저장
                output_file = os.path.join(output_dir, f"agromarket_{year}_{season_eng}.csv")
                subset.to_csv(output_file, index=False, encoding="utf-8-sig")
                print(f"파일 저장 완료: {output_file}")

# 저장 실행
output_dir = "../data/processed/agromarket_yearandseason"
save_by_year_and_season(merged_data, output_dir)

print("연도-계절별 데이터 저장 완료.")


파일 저장 완료: ../data/processed/agromarket_yearandseason\agromarket_2019_Spring.csv
파일 저장 완료: ../data/processed/agromarket_yearandseason\agromarket_2019_Summer.csv
파일 저장 완료: ../data/processed/agromarket_yearandseason\agromarket_2019_Autumn.csv
파일 저장 완료: ../data/processed/agromarket_yearandseason\agromarket_2019_Winter.csv
파일 저장 완료: ../data/processed/agromarket_yearandseason\agromarket_2020_Spring.csv
파일 저장 완료: ../data/processed/agromarket_yearandseason\agromarket_2020_Summer.csv
파일 저장 완료: ../data/processed/agromarket_yearandseason\agromarket_2020_Autumn.csv
파일 저장 완료: ../data/processed/agromarket_yearandseason\agromarket_2020_Winter.csv
파일 저장 완료: ../data/processed/agromarket_yearandseason\agromarket_2021_Spring.csv
파일 저장 완료: ../data/processed/agromarket_yearandseason\agromarket_2021_Summer.csv
파일 저장 완료: ../data/processed/agromarket_yearandseason\agromarket_2021_Autumn.csv
파일 저장 완료: ../data/processed/agromarket_yearandseason\agromarket_2021_Winter.csv
파일 저장 완료: ../data/processed/agromarket_y

# 소수점 둘째자리 처리

In [2]:
import os
import pandas as pd
from glob import glob

# 디렉토리 경로 설정
input_dir = "../data/processed/agromarket_yearandseason/"
output_dir = "../data/processed/agromarket_yearandseason_rounded/"
os.makedirs(output_dir, exist_ok=True)

# 모든 CSV 파일 읽어오기
all_files = glob(os.path.join(input_dir, "*.csv"))

# 숫자형 컬럼 정의
numeric_columns = ['총물량(kg)', '평균물량(kg)', '총금액(원)', '평균금액(원)', '총단가(원/kg)', '평균단가(원/kg)']

# 파일 처리 및 저장
for file_path in all_files:
    try:
        # 데이터 읽기
        data = pd.read_csv(file_path)
        
        # 숫자형 컬럼 소수점 둘째 자리 반올림
        data[numeric_columns] = data[numeric_columns].round(2)
        
        # 처리된 파일 저장
        file_name = os.path.basename(file_path)  # 파일 이름만 추출
        output_path = os.path.join(output_dir, file_name)
        data.to_csv(output_path, index=False, encoding="utf-8-sig", float_format="%.2f")
        
        print(f"파일 저장 완료: {output_path}")
    except Exception as e:
        print(f"파일 처리 중 오류 발생: {file_path} - {e}")


파일 저장 완료: ../data/processed/agromarket_yearandseason_rounded/agromarket_2019_Autumn.csv
파일 저장 완료: ../data/processed/agromarket_yearandseason_rounded/agromarket_2019_Spring.csv
파일 저장 완료: ../data/processed/agromarket_yearandseason_rounded/agromarket_2019_Summer.csv
파일 저장 완료: ../data/processed/agromarket_yearandseason_rounded/agromarket_2019_Winter.csv
파일 저장 완료: ../data/processed/agromarket_yearandseason_rounded/agromarket_2020_Autumn.csv
파일 저장 완료: ../data/processed/agromarket_yearandseason_rounded/agromarket_2020_Spring.csv
파일 저장 완료: ../data/processed/agromarket_yearandseason_rounded/agromarket_2020_Summer.csv
파일 저장 완료: ../data/processed/agromarket_yearandseason_rounded/agromarket_2020_Winter.csv
파일 저장 완료: ../data/processed/agromarket_yearandseason_rounded/agromarket_2021_Autumn.csv
파일 저장 완료: ../data/processed/agromarket_yearandseason_rounded/agromarket_2021_Spring.csv
파일 저장 완료: ../data/processed/agromarket_yearandseason_rounded/agromarket_2021_Summer.csv
파일 저장 완료: ../data/processed/agro